In [5]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as mpl
import geopandas as gpd
from coord_convert.transform import wgs2gcj, wgs2bd, gcj2wgs, gcj2bd, bd2wgs, bd2gcj 
from shapely.geometry import Point,Polygon,shape
import shapely.geometry
import sys
sys.path.append(r'./packages_rely/')
import plot_map
import json
import osmnx as ox
from scipy.spatial.distance import pdist
from scipy import spatial
import math
from math import radians, cos, sin, asin, sqrt
import osmnx as ox
import folium
import os
import copy
plt.rc('font',family='Times New Roman')

# 1.数据读取及流量统计

In [6]:
# 电警与交叉口映射
# 设备信息表1
dev_df = pd.read_excel("./数据/distinct_camera.xlsx")
dev_df.dropna(how='all',inplace=True,axis=0)
dev_df.dropna(how='all',inplace=True,axis=1)
# 将电警卡口的经纬度转化为84坐标系
def conv284(ser):
    return gcj2wgs(ser[0],ser[1])
    
dev_df['lnglat84'] = dev_df[['citydogcj02lng','citydogcj02lat']].apply(conv284 , axis=1)
dev_df['lng84'] = dev_df[['lnglat84']].applymap(lambda x:x[0])
dev_df['lat84'] = dev_df[['lnglat84']].applymap(lambda x:x[1])
# 读取网络
G = ox.io.load_graphml(filepath='./数据/基于osmnx的路网爬取/graph84坐标系.graphml')
# 将交叉口与电警做映射，算法是利用直线距离最近来匹配
nn, dist = ox.distance.nearest_nodes(G, dev_df['lng84'], dev_df['lat84'], return_dist=True)
dev_df['nodeid'] = nn ; dev_df['dis2node'] = dist
# 将电警与交叉口的映射可视化
dev_df = dev_df[dev_df['dis2node'] < 100]
dev_df.reset_index(drop=True,inplace=True)
dev_df.head(2)

,camera_idxx,camera_addr,in_time_latest,turn_dir,camera_type,camera_service_ip,channel_id,channel_num,lat,lng,...,citydogcj02lat,citydogcj02lng,camera_node,node_gcj02lat,node_gcj02lng,lnglat84,lng84,lat84,nodeid,dis2node
0,33010900001320130299xx,风情大道-晨晖路南向北1,2018-11-01 11:28:01.362,南向北,枪机,33.90.129.76,1000053.0,15.0,30.142625,120.251083,...,30.136641,120.244824,风情大道-晨晖路,30.137144,120.245001,"(120.24035064995653, 30.139100201406947)",120.240351,30.139100,483,51.412587
1,33010900001320130298xx,风情大道-晨晖路南向北2,2018-11-01 11:28:31.935,南向北,枪机,33.90.129.76,1000053.0,14.0,30.142625,120.251083,...,30.136672,120.244855,风情大道-晨晖路,30.137144,120.245001,"(120.2403811675365, 30.139130718984944)",120.240381,30.139131,483,47.168040


In [7]:
# cam_df = dev_df.groupby(['nodeid'])[['camera_id']].count().reset_index().rename(columns={'camera_id':'cam_num'})
# cam_df.head(2)

In [8]:
df_all = pd.DataFrame([])
for date in ['0318','0319','0320','0321','0322']:
    print("{0}日期正在被处理.".format(date))
    # 读取萧山区的电警数据
    path = './数据/萧山市车牌识别数据/'
    df = pd.read_csv(path+date+".csv")
    df.drop(columns=['Unnamed: 0'],inplace=True)
    df.rename(columns={'0':'car_num','1':'cap_date','2':'type','3':'dev_id','4':'dir','5':'road_id','6':'turn_id'},inplace=True)
    # 解决车辆在同一时间同一地点被重复检测问题
    len1 = len(df)
    df = df.drop_duplicates(subset=['car_num' , 'cap_date'])
    print("重复数据占比为{0}".format((len1-len(df)) / len1))
    # 统计各个交叉口的分时段流量
    #将电警表和过车数据表merge起来
    df = pd.merge(df[['car_num','cap_date','dev_id']],dev_df[['camera_id','nodeid']],how = 'inner',left_on='dev_id',right_on='camera_id')
    df.drop(columns=['camera_id'] , inplace=True)
    # 早晚高峰给予更高的权重
    # 统计早晚高峰
    df['ana_t'] = df[['cap_date']].applymap(lambda x:x[11:16])
    df_peak = df[((df['ana_t'] >= '06:30')&(df['ana_t'] <= '10:00')) | ((df['ana_t'] >= '17:00')&(df['ana_t'] <= '19:30'))]
    df_peak = df_peak.groupby(['nodeid'])[['car_num']].count().reset_index().rename(columns = {'car_num':'peakfreq'})
    # 统计平峰时段
    df_nonpeak = df[(df['ana_t']<'06:30')|(df['ana_t']>'19:30')|((df['ana_t'] >'10:00')&(df['ana_t']<='17:00'))]
    df_nonpeak = df_nonpeak.groupby(['nodeid'])[['car_num']].count().reset_index().rename(columns = {'car_num':'nonpeakfreq'})
    df = pd.merge(df_peak,df_nonpeak,how='outer',on='nodeid')
    df[date] = 0.8*df['peakfreq'] + 0.2*df['nonpeakfreq']
    df = df[['nodeid',date]]
    # 计算平均电警被检测频次
#     df = pd.merge(df , cam_df , how='inner',on='nodeid')
#     df[date] = df[date] / df['cam_num']
#     df.drop(columns=['cam_num'] , inplace=True)
    if len(df_all) == 0:
        df_all = copy.deepcopy(df)
    else:
        df_all = pd.merge(df_all , df , how='outer' , on='nodeid')
df_all

0318日期正在被处理.
重复数据占比为0.09892662759220844
0319日期正在被处理.
重复数据占比为0.10318317668995049
0320日期正在被处理.
重复数据占比为0.1054003294703309
0321日期正在被处理.
重复数据占比为0.09704732091897067
0322日期正在被处理.
重复数据占比为0.1010674548931182


,nodeid,0318,0319,0320,0321,0322
0,21,4553.2,5034.4,6081.6,5993.6,1827.4
1,22,4725.2,4722.6,4832.0,4678.4,4869.4
2,24,10084.8,10426.0,10205.4,10037.0,10610.2
3,25,8331.6,9943.6,11312.6,11554.6,3420.2
4,26,28580.0,28721.2,29347.2,28059.2,30032.6
...,...,...,...,...,...,...
217,991,6373.6,6613.2,6667.4,6465.6,6836.4
218,1012,5035.0,6040.8,6237.8,6271.8,6673.8
219,600,NaN,4.6,2.0,1.6,2.8
220,27,NaN,NaN,7943.8,12176.6,13766.4


In [9]:
# 将各天的数据求均值
df_all['freq'] = df_all[['0318','0319','0320','0321','0322']].mean(axis=1)
df_all = df_all[['nodeid','freq']]
df_all.to_csv("./数据/结点重要性.csv",index=False,encoding='gbk')
df_all.head(2)

,nodeid,freq
0,21,4698.04
1,22,4765.52
